<a href="https://colab.research.google.com/github/jsanmigimeno/FYP-Pix2Pix/blob/master/notebooks/FYP_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive
By default, generated files are designed to be saved to a folder named *FYP* on a mounted Google Drive account. If it is not desired to use a Google Drive account, change the variable *rootDir* to point to a valid directory where to save the generated files.

In [0]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# Default dir
rootDir = "/content/gdrive/My\ Drive/FYP"
!mkdir rootDir

# Clone Pix2Pix and install requirements
Download the latest available adapted Pix2Pix implementation, and install any requirements.

In [0]:
# Clone code
branch = 'master'
print("Cloning branch %s" % branch)
!git clone --single-branch --branch $branch https://github.com/jsanmigimeno/FYP-Pix2Pix.git
  
# Install requirements
%cd FYP-Pix2Pix
!pip install -r requirements.txt

## Update Local Code
If desired, download a new version of the code without deleting any downloaded datasets

In [0]:
print("Updating code from branch %s" % branch)
% cd ..
# Move dataset folder outside code directory
!mv ./FYP-Pix2Pix/DatasetsData ./
# Remove code directory
!rm -rf FYP-Pix2Pix/
# Clone new code
!git clone --single-branch --branch $branch https://github.com/jsanmigimeno/FYP-Pix2Pix.git
# Move dataset folder back into code directory
!mv ./DatasetsData ./FYP-Pix2Pix
%cd FYP-Pix2Pix

# Get dataset

In [0]:
# Download dataset
dataset = 'DSLog'  # 'Downscaled_All_JPEG' or 'DSLog'

!mkdir ./DatasetsData/
!python ./scripts/DatasetURLs.py -p './DatasetsData/' -d $dataset

# Train

In [0]:
def paramsListToString(paramsList):
  m = ""
  for param in paramsList:
    m += str(param) + " "
  return m

In [0]:
# Parameters
trainingName = 'trainingName'

params = [
    # Basic model parameters
    "--model", "pix2pix",                           # Model pix2pix
    "--display_id", 0,                              # Disable visdom
    "--checkpoints_dir", rootDir,                   # Path to save checkpoints and logs
    "--dataroot", './DatasetsData/' + dataset,      # Dataset path './Downscaled_All_JPEG/'
    "--name", trainingName,                         # Training name
    "--use_validation",                             # Use validation whilst training
    
    # Dataset parameters
    "--dataset_mode", "aligned",                    # Aligned dataset mode (AB concatenated)
    "--load_size", 2120,                            # Resolution to which load images with (width) (default: 2120)
    "--crop_size", 256,                             # Size to which to crop the images for training (default: 256)
    "--preprocess", "crop",                         # Type of preprocessing (default: crop, NOTE THAT LOAD SIZE IS IGNORED WITH THIS SETTING)
    
    # Log parameters
    "--save_epoch_freq", 10,                        # Frequency at which to save the model (default: 5)
    "--save_opt_freq", 10,                          # Frequency at which to save the optimiser (default: 20)
    "--niter", 100,                                 # Number of iterations at starting learning rate (default: 100)
    "--niter_decay", 100,                           # Number of iterations to linearly decay learning rate to zero (default: 100)
    
    # Loss parameters
    "--lambda_GAN", 1,                              # GAN loss multiplier (default: 1)
    "--lambda_L1", 100,                             # L1 coefficient loss (default: 100)
    "--lambda_desc", 100,                           # Matching coefficient loss (default: 0 - i.e. disabled)
    #"--siamese_descriptor",                        # Use siamese descriptor
    #"--per_channel_descriptor"                     # Use per RGB channel descriptors
    #"--descriptor", "HardNet"                      # Type of descriptor to use for the loss
    #"--non_empty_patches_only"
]

# Continue training from saved epoch
continueTrain = False                               # Whether to resume training (default: False)
epoch = 0                                           # Epoch from which to resume training

contTrainParams = [
    # Resume training parameters
    "--continue_train",                             # Resume training
    "--epoch", epoch,                               # Epoch from which to resume training
    "--epoch_count", epoch + 1,                     # Epoch count from which to resume training 
]

if continueTrain:
  params = params + contTrainParams

paramsString = paramsListToString(params)

!python train.py $paramsString